In [ ]:
%pip install aiohttp

# Unified Workflow Setup

This notebook serves as the universal setup for all three document processing workflows:

1. **Document Submission Service** - File monitoring and API submission
2. **Document Result Polling Service** - API polling and result download  
3. **Document Processing Orchestrator** - Bronze/Silver/Gold data processing

## Key Functions:
- **Configuration Management** - Loads and validates pipeline configuration from YAML
- **Infrastructure Validation** - Checks schemas, tables, and API connectivity
- **Task Value Setup** - Sets comprehensive task values for all downstream workflow tasks
- **Unified Configuration** - Single source of truth for all workflow parameters


## Imports

In [ ]:
# Import required libraries
from pyspark.sql import functions as F
from pyspark.sql.types import *
import json


## ConfigManager

TODO: Move to framework once packaging issue is resolved

In [ ]:
"""
Configuration management utilities for PDF processing framework.

This module provides utilities for loading, validating, and managing
configuration files used throughout the PDF processing pipeline.
"""

import yaml
import os
from typing import Dict, Any, List, Optional, Union
from pathlib import Path


class ConfigManager:
    """
    Configuration manager for handling YAML and JSON configuration files.
    
    Provides methods for:
    - Loading configuration from files
    - Validating configuration structure
    - Merging configurations
    - Environment variable substitution
    """
    
    def __init__(self, config_path: Optional[Union[str, Path]] = None):
        """
        Initialize configuration manager.
        
        Args:
            config_path: Path to configuration file
            config_dict: Configuration dictionary (alternative to file)
            
        Raises:
            ValueError: If neither config_path nor config_dict is provided
        """
        if config_path:
            self.config = self.load_config_file(config_path)
            self.config_path = str(config_path)
        else:
            raise ValueError("Either config_path or config_dict must be provided")
        
        # Substitute environment variables
        # self._substitute_environment_variables()

    @staticmethod
    def get_file_extension(file_path: Union[str, Path]) -> str:
        """
        Get file extension (without dot).
        
        Args:
            file_path: Path to file
            
        Returns:
            File extension in lowercase
        """
        return Path(file_path).suffix.lower().lstrip('.')
    
    @staticmethod
    def load_config_file(file_path: Union[str, Path]) -> Dict[str, Any]:
        """
        Load configuration from YAML or JSON file.
        
        Args:
            file_path: Path to configuration file
            
        Returns:
            Configuration dictionary
            
        Raises:
            FileNotFoundError: If file doesn't exist
            ValueError: If file format is not supported
        """
        extension = ConfigManager.get_file_extension(file_path)
        
        if extension in ['yaml', 'yml']:
            with open(file_path, 'r', encoding='utf-8') as f:
                return yaml.safe_load(f) or {}
        else:
            raise ValueError(f"Unsupported configuration file format: {extension}")
    
    def get(self, key: str, default: Any = None) -> Any:
        """
        Get configuration value with optional default.
        
        Args:
            key: Configuration key (supports dot notation for nested keys)
            default: Default value if key is not found
            
        Returns:
            Configuration value or default
        """
        return self._get_nested_value(self.config, key, default)
        
    @staticmethod
    def _get_nested_value(config: Dict[str, Any], key: str, default: Any = None) -> Any:
        """
        Get nested configuration value using dot notation.
        
        Args:
            config: Configuration dictionary
            key: Key with dot notation (e.g., 'task.params.value')
            default: Default value if key is not found
            
        Returns:
            Configuration value or default
        """
        keys = key.split('.')
        value = config
        
        for k in keys:
            if isinstance(value, dict) and k in value:
                value = value[k]
            else:
                return default
        
        return value
        
    def to_dict(self) -> Dict[str, Any]:
        """
        Get configuration as dictionary.
        
        Returns:
            Configuration dictionary
        """
        return self.config.copy()
    

    def get_pipeline_steps(config: dict, pipeline_name: str, document_format: str, step_name: str) -> dict:
        """
        Return the steps dictionary for a given pipeline and document format.
        """
        step_list = config.get(pipeline_name, {}).get(document_format, [])
        step_config = ConfigManager.get_step_config(step_list, step_name)
        return step_config


    @staticmethod
    def get_step_config(config: list, step_name: str) -> dict:
        """
        Fetch all variables for a given step from the workflow config.

        Args:
            config (list): List of step dictionaries.
            step_name (str): Step name to search for.

        Returns:
            dict: Config dictionary for the step, or {} if not found.
        """
        for step in config:
            if step.get("step") == step_name:
                return step
        return {}

   


## Load Config Manager 

In [ ]:


# Read pipeline configuration from YAML using ConfigManager
pipeline_config_path = dbutils.widgets.get("pipeline_config_path")
environment = dbutils.widgets.get("environment")
print(f"📋 Loading pipeline configuration from: {pipeline_config_path} for environment {environment}")

# Get job tracking parameters (passed from job YAML)
job_id = dbutils.widgets.get("job_id")
job_run_id = dbutils.widgets.get("job_run_id")

# Use ConfigManager to load and manage configuration
try:
    config_manager = ConfigManager(config_path=pipeline_config_path)
    config = config_manager.to_dict()
    
    # Use ConfigManager methods for accessing nested configuration
    globals_config = config_manager.get("globals", {})
    services_config = config_manager.get("services", {})
    pipelines_config = config_manager.get("pipelines", {})
    unified_steps_config = config_manager.get("unified_steps", {})
    
    print(f"✓ Configuration sections loaded:")
    print(f"   - Globals: {len(globals_config)} keys")
    print(f"   - Services: {len(services_config)} keys") 
    print(f"   - Pipelines: {pipelines_config} keys")
    print(f"   - Unified Steps: {unified_steps_config} keys")
except FileNotFoundError:
    print(f"File not found at path: {pipeline_config_path}")
    raise
except Exception as e:
    print(f"Error loading configuration file: {e}")
    raise

print(f"✓ Configuration loaded using ConfigManager:")




## Set Task variables

In [ ]:


# Store configuration as task values for other workflow tasks using ConfigManager
print("📤 Setting task values for downstream workflows...")

dbutils.jobs.taskValues.set(key="environment", value=environment)
dbutils.jobs.taskValues.set(key="databricks_job_id", value=job_id)
dbutils.jobs.taskValues.set(key="databricks_job_run_id", value=job_run_id)

# Use ConfigManager to serialize configuration sections
dbutils.jobs.taskValues.set(key="globals_config", value=json.dumps(globals_config))
dbutils.jobs.taskValues.set(key="services_config", value=json.dumps(services_config))
dbutils.jobs.taskValues.set(key="pipelines_config", value=json.dumps(pipelines_config))
dbutils.jobs.taskValues.set(key="unified_steps_config", value=json.dumps(unified_steps_config))

# Set specific configuration values that are commonly used
dbutils.jobs.taskValues.set(key="catalog", value=config_manager.get("globals.catalog"))
dbutils.jobs.taskValues.set(key="schema_prefix", value=config_manager.get("globals.schema_prefix"))
dbutils.jobs.taskValues.set(key="checkpoint_base_location", value=config_manager.get("globals.checkpoint_base_location"))

# Set synthetic data flag for vector processing
synthetic_data_flag = config_manager.get("globals.synthetic_data", "true")
dbutils.jobs.taskValues.set(key="synthetic_data", value=synthetic_data_flag)

print("✅ UNIFIED WORKFLOW SETUP COMPLETED WITH CONFIGMANAGER")
print(f"📊 Configuration managed by framework ConfigManager:")
print(f"  - Global settings: {len(globals_config)} items")
print(f"  - Services: {len(services_config)} items")
print(f"  - Pipelines Config: {len(pipelines_config)} items")
print(f"  - Unified Steps Config: {len(unified_steps_config)} items")
print(f"  - Synthetic Data Mode: {synthetic_data_flag}")
print(f"  - Configuration validation: ✅ Passed")
print(f"")
print(f"🔧 Task values set for all workflows:")
print(f"")
print(f"🚀 This setup enables all three workflows:")
print(f"  1. Document Submission Service")
print(f"  2. Document Result Polling Service") 
print(f"  3. Document Processing Orchestrator")


TODO: Validate all services, schemas required for application to work